In [1]:
import websocket
import json
# import pprint
#import talib
import numpy
import pandas as pd
import src.credentials
import src.binance_functions

# import config
from binance.client import Client
from binance.enums import *

SOCKET = "wss://stream.binance.com:9443/ws/btcbrl@kline_1m"

RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 65
TRADE_SYMBOL = 'BTCBRL'
TRADE_QUANTITY = 0.05

client = Client(credentials.app_key, credentials.app_secret, tld='us')


ModuleNotFoundError: No module named 'src'

In [3]:
response = binance_functions.send_signed_request("GET", "/api/v3/account")

GET https://api.binance.com/api/v3/account?timestamp=1708379876078&signature=42990f8944a50d8781e4ffcf99c5095a48809fbc99d762268fb0749583b3630c


In [4]:
response

{'makerCommission': 10,
 'takerCommission': 10,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00100000',
  'taker': '0.00100000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1708192826264,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNT', 'free': '0.00000000', 'locke

In [3]:
df_balances = pd.DataFrame(response['balances'])
df_balances['free'] = df_balances['free'].astype(float)
df_balances['locked'] = df_balances['locked'].astype(float)
df_balances[(df_balances['free'] + df_balances['locked']) > 0]


,asset,free,locked
208,BRL,50.0,0.0


In [4]:
brl_balance = int(df_balances[df_balances['asset'] == 'BRL']['free'])
type(int(brl_balance))

int

* DEFINIR TRADE_QUANTITY
* GUARDAR TRANSAÇÕES EM ALGUM DATAFRAME


In [ ]:
closes = []
in_position = False

df_orders = pd.DataFrame(columns=[''])

client = Client(credentials.API_KEY, credentials.API_SECRET, tld='us')

baseline = 50

def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET):
    try:
        print("sending order")
        # order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        print(order)
        
    except Exception as e:
        print("an exception occured - {}".format(e))
        return False

    return True

    
def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global closes, in_position
    
    # print('received message')
    json_message = json.loads(message)
    # pprint.pprint(json_message)

    candle = json_message['k']

    is_candle_closed = candle['x']
    close = candle['c']

    if is_candle_closed:
        print("candle closed at {} ' --- closes: ', {}".format(close, len(closes)))
        closes.append(float(close))
        print("closes")
        print(' --- closes: ', len(closes))

        if len(closes) > RSI_PERIOD:
            np_closes = numpy.array(closes)
            rsi = talib.RSI(np_closes, RSI_PERIOD)
            
            last_rsi = rsi[-1]
            print("the current rsi is {}".format(last_rsi))


            if last_rsi > RSI_OVERBOUGHT and in_position:
                
                if in_position:
                
                    # get BTC free balance
                    btc_balance = binance_functions.get_balance('BTC')
                    
                    # convert BTC balance to BRL
                    btcbrl_balance = close * btc_balance

                    # get profit compared to baseline
                    profit_value = btcbrl_balance - baseline
                    
                    if profit_value > 0:

                        # convert profit in BRL to BTC
                        TRADE_QUANTITY = profit_value / close
                        
                        # create sell order
                        order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)                    
                        if order_succeeded:
                            in_position = False 

                        in_position = False
                    
                    else:

                        print('In position to sell, but there''s no profit over the baseline')
                else:
                    # df.at[i, 'position'] = 'DOI'
                    print('In position to sell, but don''t on it yet')
                    
            elif last_rsi < RSI_OVERSOLD:
                
                if in_position:
                    print('In position to buy, but already have it')
                    # df.at[i, 'position'] = 'HOLD'
                else:
                    
                    # get BRL free balance
                    brl_balance = binance_functions.get_balance('BRL')

                    if brl_balance > 0:
                
                        # convert BRL balance to BTC
                        TRADE_QUANTITY = brl_balance / close
                        
                        # create buy order
                        order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)
                        if order_succeeded:
                            in_position = True

                        # df.at[i, 'position'] = 'BOUGHT'
                        in_position = True

                    else:
                        
                        print('In position to buy, but don''t have enough balance')
            else:
                #RSI entre overbought e oversold
                print('No position to take')
                # df.at[i, 'position'] = 'HOLD'




In [ ]:
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

In [ ]:
from src import credentials

KEY = credentials.app_key
KEY

In [9]:
import websocket
import json
import numpy as np
import pandas as pd
import credentials as cd
import binance_functions

from binance.client import Client, BinanceAPIException
from binance.enums import *

price_list = []
total_vendas = 0
df_prices = pd.DataFrame(columns =['close', 'var', 'positivo'
                                       , 'negativo', 'media_positivos', 'media_negativo' 
                                       ,'position','quantity','symbol'
                                       ,'btc_balance','btcbrl_balance'
                                       ,'profit_value' ,'in_position_after'])
SIMULATE_FLG = False

In [2]:
def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET):
    if not SIMULATE_FLG:
        try:
            print("sending order")
            order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
            print(order)
            
        except Exception as e:
            print("an exception occured - {}".format(e))
            return False
        
    return True

## PARA TESTAR A LÓGICA SEM CHAMAR WEBSOCKET
def random_test(df_prices, closes, RSI_PERIOD, RSI_OVERBOUGHT, RSI_OVERSOLD, in_position):
    
    close_lst = np.random.randint(240000, 250000, size=(1000))
    
    for close in close_lst:
        
        in_position = process_new_price(close, df_prices, close_lst, RSI_PERIOD, RSI_OVERBOUGHT, RSI_OVERSOLD, in_position)


def calculate_rsi(close_prices, rsi_period):
    """
    Calcula o Índice de Força Relativa (RSI) com base nos preços de fechamento e no período especificado.
    """
    # Calcula as variações dos preços de fechamento
    price_diff = close_prices.diff()

    # Separa as variações em positivas e negativas
    gain = price_diff.where(price_diff > 0, 0)
    loss = -price_diff.where(price_diff < 0, 0)

    # Calcula as médias móveis dos ganhos e das perdas
    avg_gain = gain.rolling(window=rsi_period).mean()
    avg_loss = loss.rolling(window=rsi_period).mean()

    # Calcula o RSI
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.iloc[-1]  # Retorna apenas o último valor do RSI

def process_new_price(close, df_prices, price_list, RSI_PERIOD, RSI_OVERBOUGHT, RSI_OVERSOLD, in_position):
    """
    Processa um novo preço recebido, atualiza o DataFrame de preços e verifica sinais de negociação com base no RSI.
    """
    try:
        if not SIMULATE_FLG:
            price_list.append(float(close))

        # Adiciona o preço de fechamento ao DataFrame
        df_prices.loc[len(df_prices)] = {'close': float(close), 'RSI': None, 'compra': False, 'venda': False}

        if len(price_list) >= RSI_PERIOD:
            # Calcula o RSI para a última linha
            rsi = calculate_rsi(df_prices['close'], RSI_PERIOD)

            # Atualiza o DataFrame com o valor de RSI calculado
            df_prices.at[len(df_prices) - 1, 'RSI'] = rsi

            # Calcula outras métricas
            df_prices['var'] = df_prices['close'].pct_change()
            df_prices['positivo'] = df_prices['var'].apply(lambda x: x if x > 0 else 0)
            df_prices['negativo'] = df_prices['var'].apply(lambda x: abs(x) if x < 0 else 0)
            df_prices['media_positivos'] = df_prices['positivo'].rolling(window=RSI_PERIOD).mean()
            df_prices['media_negativo'] = df_prices['negativo'].rolling(window=RSI_PERIOD).mean()

            # Verifica os sinais de negociação com base no RSI e na variável in_position
            if rsi > RSI_OVERBOUGHT and in_position:
                print(f'RSI: {rsi} > RSI_OVERBOUGHT: {RSI_OVERBOUGHT}. Vendendo...')
                
                # get BTC free balance
                btc_balance = binance_functions.get_balance('BTC')
                print('btc_balance: ', btc_balance)

                # convert BTC balance to BRL
                btcbrl_balance = close * btc_balance
                print('btcbrl_balance: ', btcbrl_balance)

                # get profit compared to baseline
                profit_value = btcbrl_balance - baseline
                print('profit_value: ', profit_value)

                # Get BRL balance before order
                brl_balance_before = binance_functions.get_balance('BRL')
                
                # convert profit in BRL to BTC
                TRADE_QUANTITY = btc_balance #profit_value / close

                if profit_value > 0:
                    
                    df_prices['in_position_before'] = in_position
                    # create sell order
                    order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)                    
                    if order_succeeded:
                        print('VENDEU!')
                        total_vendas += 1
                        in_position = False 
                        
                        if total_vendas == 4:
                            ws.close()

                    # Get BRL balance after order
                    brl_balance_after = binance_functions.get_balance('BRL')
                    
                    print(f'RSI: {rsi} > RSI_OVERBOUGHT: {RSI_OVERBOUGHT}. Vendendo...')
                    df_prices.at[len(df_prices) - 1, 'hold'] = False
                    df_prices.at[len(df_prices) - 1, 'venda'] = True
                    df_prices['position'] = 'SELL'
                    df_prices['quantity'] = TRADE_QUANTITY
                    df_prices['symbol'] = TRADE_SYMBOL
                    df_prices['brl_balance_before'] = brl_balance_before
                    df_prices['brl_balance_after'] = brl_balance_after
                    df_prices['btc_balance'] = btc_balance
                    df_prices['btcbrl_balance'] = btcbrl_balance
                    df_prices['profit_value'] = profit_value
                    df_prices['in_position_after'] = in_position
                else:

                    print('In position to sell, but there''s no profit over the baseline')
                    df_prices['position'] = 'SIDE_SELL_NO_PROFITS'
                    df_prices['quantity'] = TRADE_QUANTITY
                    df_prices['symbol'] = TRADE_SYMBOL
                    df_prices['brl_balance_before'] = brl_balance_before
                    df_prices['btc_balance'] = btc_balance
                    df_prices['btcbrl_balance'] = btcbrl_balance
                    df_prices['profit_value'] = profit_value
                    df_prices['in_position_after'] = in_position
            
            elif rsi > RSI_OVERBOUGHT and not in_position:
                print(f'RSI: {rsi} > RSI_OVERBOUGHT: MAS NÃO ESTAMOS EM POSIÇÃO. HOLD...')
                df_prices.at[len(df_prices) - 1, 'hold'] = True
                df_prices['position'] = 'SIDE_SELL_DONT_OWN_YET'
            
            elif rsi < RSI_OVERSOLD and not in_position:
                # get BTC free balance
                btc_balance = binance_functions.get_balance('BTC')
                
                # convert BTC balance to BRL
                btcbrl_balance = close * btc_balance

                # get profit compared to baseline
                profit_value = btcbrl_balance - baseline

                # get BRL free balance
                brl_balance_before = binance_functions.get_balance('BRL')
                print('brl_balance: ', brl_balance_before)

                # convert BRL balance to BTC
                TRADE_QUANTITY = brl_balance_before / close

                if brl_balance_before > 0:
                    
                    #df_prices['position'] = 'SIDE_BUY_ALREADY_OWN'
                    print(f'Compra: {type(TRADE_QAUNTITY)} {TRADE_QAUNTITY}')
                    # create buy order
                    order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)

                    df_prices['in_position_before'] = in_position

                    if order_succeeded:
                        in_position = True

                    # Get BRL balance after order
                    brl_balance_after = binance_functions.get_balance('BRL')
                    
                    print(f'RSI: {rsi} < RSI_OVERSOLD: {RSI_OVERSOLD}. Comprando...')
                    df_prices.at[len(df_prices) - 1, 'hold'] = False
                    df_prices.at[len(df_prices) - 1, 'compra'] = True
                    df_prices['position'] = 'BUY'
                    df_prices['quantity'] = TRADE_QUANTITY
                    df_prices['symbol'] = TRADE_SYMBOL
                    df_prices['brl_balance_before'] = brl_balance_before
                    df_prices['brl_balance_after'] = brl_balance_after
                    df_prices['btc_balance'] = btc_balance
                    df_prices['btcbrl_balance'] = btcbrl_balance
                    df_prices['profit_value'] = profit_value
                    df_prices['in_position_after'] = in_position

                else:
                    
                    print(f'RSI: {rsi} < RSI_OVERSOLD: MAS NÃO TEMOS SALDO SUFICIENTE. HOLD...')
                    df_prices['position'] = 'SIDE_BUY_NO_BALANCE'
                    df_prices['quantity'] = TRADE_QUANTITY
                    df_prices['symbol'] = TRADE_SYMBOL
                    df_prices['brl_balance_before'] = brl_balance_before
                    df_prices['btc_balance'] = btc_balance
                    df_prices['btcbrl_balance'] = btcbrl_balance
                    df_prices['profit_value'] = profit_value
                    df_prices['in_position_after'] = in_position
            
            elif rsi < RSI_OVERSOLD and in_position:
                print(f'RSI: {rsi} < RSI_OVERSOLD: MAS JÁ ESTAMOS EM POSIÇÃO. HOLD...')
                df_prices.at[len(df_prices) - 1, 'hold'] = True

            else:
                print(f'RSI: {rsi} entre os indicadores: HOLD... ')
                df_prices.at[len(df_prices) - 1, 'hold'] = True
            
            # WRITES LAST LINE ON CSV
            last_close = df_prices.iloc[-1:]
            if len(closes) == (RSI_PERIOD + 1):
                last_close.to_csv('positions.csv', header=True, index=False)
                print('--- CRIOU O CSV ---')
            else:
                last_close.to_csv('positions.csv', mode='a', header=False, index=False)
        
        return in_position
    except Exception as e:
        print(e)
    
def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    """
    Função de callback chamada quando uma nova mensagem é recebida pelo WebSocket.
    Processa a mensagem JSON para obter o preço de fechamento e chama a função de processamento de preço.
    """
    try:
        global in_position
        json_message = json.loads(message)
        candle = json_message['k']
        is_candle_closed = candle['x']
        close = candle['c']
        
        if not is_candle_closed:
            in_position = process_new_price(close, df_prices, price_list, RSI_PERIOD, RSI_OVERBOUGHT, RSI_OVERSOLD, in_position)

    except Exception as e:
        print(e)


In [52]:
SOCKET = "wss://stream.binance.com:9443/ws/btcbrl@kline_1m"

RSI_PERIOD = 22
RSI_OVERBOUGHT = 75
RSI_OVERSOLD = 60
TRADE_SYMBOL = 'BTCBRL'
TRADE_QUANTITY = 0.05

closes = []
in_position = False    

client = Client(cd.API_KEY, cd.API_SECRET, tld='us')

# set profit baseline to drive sell/buy operations
baseline = 50

# start app
print(' --- Hello World! We are starting! ---')
if not SIMULATE_FLG:
    try:
        print('Declare WebSocket')
        ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
        print(f'Run WebSocket - Start - {ws}')
        ws.run_forever()
        print('Run WebSocket - End')
    except BinanceAPIException as e:
        print(f'Error: {e}')
else:
    random_test(df_prices, closes, RSI_PERIOD, RSI_OVERBOUGHT, RSI_OVERSOLD, in_position)

print('End script')

 --- Hello World! We are starting! ---
Declare WebSocket
Run WebSocket - Start - <websocket._app.WebSocketApp object at 0x1419b4590>
opened connection


GET https://api.binance.com/api/v3/account?timestamp=1710295632012&signature=03648373e5be0985500b21704cfeb051620ef7367e1c1af0f13a0b861dc3071c
unsupported operand type(s) for -: 'str' and 'int'
GET https://api.binance.com/api/v3/account?timestamp=1710295634471&signature=a2bfe29367bac92b4a62c21f1d5269d9aa8421b4ee7cfcfb2fd59950e52f7496
unsupported operand type(s) for -: 'str' and 'int'
GET https://api.binance.com/api/v3/account?timestamp=1710295638003&signature=671f18f4b44b38cfae7f87fe9f15ee9931a462f7ca59ea81d82ced4d52ae7f09
unsupported operand type(s) for -: 'str' and 'int'
GET https://api.binance.com/api/v3/account?timestamp=1710295644010&signature=add06393a6c2d3fe96575cdc7824dbf39eda692dd316c4d335c4dd18279f863f
unsupported operand type(s) for -: 'str' and 'int'
GET https://api.binance.com/api/v3/account?timestamp=1710295645934&signature=a9bdbaf85bd41e49b80d59c4c26064c4d241392fade32d3fee9cef5604c20fdd
unsupported operand type(s) for -: 'str' and 'int'
GET https://api.binance.com/api/v3/

In [19]:
df_prices.to_excel('positions.xlsx', index=False)

In [ ]:
client = Client(cd.API_KEY, cd.API_SECRET, tld='us')
client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)

In [7]:
payload

{'wallet': 'SPOT_WALLET',
 'fromAsset': 'BTC',
 'toAsset': 'BRL',
 'fromAmount': None,
 'toAmount': 5}

In [10]:
fromAsset = 'BTC'
toAsset = 'BRL'
fromAmount=None
toAmount=5
#def get_balance(fromAsset, toAsset, fromAmount=None, toAmount=None): 

# validate type param
#assert type in ['free','locked'], 'Invalid balance type'

payload = {
            "wallet": "SPOT_WALLET",
            "fromAsset": fromAsset,
            "toAsset": toAsset,
            "fromAmount": fromAmount,
            "toAmount": toAmount
        }

# get account balances list
response = binance_functions.send_signed_request("POST", "/bapi/margin/v1/private/new-otc/execute-quote", payload)
# response = binance_functions.send_signed_request("POST", "/binancepay/openapi/otc-portal/get-quote", payload)

# treat and filter balances greater than 0
#df_balances = 
pd.DataFrame(response['data'])

POST https://api.binance.com/bapi/margin/v1/private/new-otc/execute-quote?wallet=SPOT_WALLET&fromAsset=BTC&toAsset=BRL&fromAmount=None&toAmount=5&timestamp=1713827959895&signature=ef388f15f7677516ee4a07ec97a42cabe4ca63c2e04968d65444e228d01599e7


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [12]:
# get account balances list
response = binance_functions.send_signed_request("GET", "/api/v3/account")

# treat and filter balances greater than 0
df_balances = pd.DataFrame(response['balances'])
df_balances['free'] = df_balances['free'].astype(float)
df_balances['locked'] = df_balances['locked'].astype(float)
df_balances[(df_balances['free'] + df_balances['locked']) > 0]

# return balance from the symbol and type selected
int(df_balances[df_balances['asset'] == 'BTC'][type])

GET https://api.binance.com/api/v3/account?timestamp=1713828217787&signature=7c30db98527a27f8dd22339860be48048c90e8cdb515e69f242ea2c7506195f9


KeyError: <class 'pandas.core.frame.DataFrame'>